In [1]:
import pickle
#Load Dictionary
with open('transcripts_timestamps.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

In [2]:
def combine_text_fragments(documents, chunk_size=3):
    combined_docs = []
    current_chunk = {"Title": documents[0]['Title'], "Text": "", "TimeStamp": documents[0]['TimeStamp']}
    
    for i, doc in enumerate(documents):
        if i % chunk_size == 0 and i != 0:
            combined_docs.append(current_chunk)
            current_chunk = {"Title": doc['Title'], "Text": "", "TimeStamp": doc['TimeStamp']}
        
        current_chunk['Text'] += f" {doc['Text']}"
    
    combined_docs.append(current_chunk)
    return combined_docs

# Example usage
documents = transcripts
combined_documents = combine_text_fragments(documents)

In [4]:
len(combined_documents)

3356

In [5]:
len(transcripts)

10068

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Or any suitable model

def generate_embeddings(docs):
    for doc in docs:
        doc['embedding'] = model.encode(doc['Text'])
    return docs

combined_documents = generate_embeddings(combined_documents)


/home/gary/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-08 13:31:09.521420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 13:31:09.645960: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 13:31:09.646626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-08 13:31:09.836320: I tensorflow/core/platform/cpu_feature_gua

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

def index_documents(docs, index_name="transcripts"):
    es = Elasticsearch("http://localhost:9200")

    actions = [
        {
            "_index": index_name,
            "_source": doc
        }
        for doc in docs
    ]
    bulk(es, actions)

index_documents(combined_documents)


/tmp/ipykernel_169127/4262659160.py:14: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  bulk(es, actions)


In [14]:
import numpy as np

def retrieve_documents_question(query, index_name="transcripts", max_results=28):
    es = Elasticsearch("http://localhost:9200")
    
    # Generate query embedding
    query_embedding = model.encode(query)
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "should": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["Title", "Text^4"],
                            "type": "most_fields"
                        }
                    },
                    {
                        "script_score": {
                            "query": {"match_all": {}},
                            "script": {
                                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                "params": {"query_vector": query_embedding}
                            }
                        }
                    }
                ]
            }
        },
        "_source": ["Title", "Text", "TimeStamp"]
    }
    
    try:
        response = es.search(index=index_name, body=search_query)
        result_docs = [hit['_source'] for hit in response['hits']['hits']]
        return result_docs
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

# Example usage
query = "introduction to LLM"
# query = generate_embeddings(query)
results = retrieve_documents_question(query)
for result in results:
    print(result)


Error retrieving documents: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')


In [15]:
# Example usage
query = "introduction to LLM"
# query = generate_embeddings(query)
results = retrieve_documents_question(query)
for result in results:
    print(result)

Error retrieving documents: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')


In [12]:
from elasticsearch import Elasticsearch, exceptions

def retrieve_documents_question(query, index_name="transcripts", max_results=28):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["Title", "Text^4"],
                "type": "most_fields"
            }
        }
    }
    
    try:
        response = es.search(index=index_name, body=search_query)
        result_docs = [hit['_source'] for hit in response['hits']['hits']]
        return result_docs
    except exceptions.BadRequestError as e:
        print(f"Bad request error: {e.info}")
        return []
    except exceptions.NotFoundError:
        print(f"Index {index_name} not found.")
        return []
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

# Test the function
query = "introduction to LLM"
results = retrieve_documents_question(query)
for result in results:
    print(result)


{'Title': 'llm zoomcamp 4.1 - introduction to monitoring answer quality', 'Text': " but for today we're just going to do an extended introduction like what are the topics that we are going to cover and", 'TimeStamp': 82.84, 'embedding': [-0.021802693605422974, -0.007218487095087767, 0.07080982625484467, -0.004802611656486988, 0.07323632389307022, 0.0588289238512516, -0.030148493126034737, 0.00541073689237237, -0.03981546685099602, -0.01563945598900318, -0.06520326435565948, 0.11812259256839752, -0.11990741640329361, -0.0247196014970541, 0.07731315493583679, -0.08549807965755463, 0.020124631002545357, -0.08097734302282333, -0.0004959985963068902, 0.03733066841959953, 0.03677303344011307, 0.043684136122465134, 0.038075610995292664, -0.016855932772159576, -0.08437169343233109, 0.014866513200104237, 0.016889432445168495, -0.009174371138215065, 0.06397146731615067, 0.05278242751955986, -0.028715800493955612, 0.07357572764158249, 0.035061828792095184, 0.01186402142047882, -0.0374901778995990

/tmp/ipykernel_169127/1339247122.py:18: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.search(index=index_name, body=search_query)
